# Helper Function List

This is a Helper function list used to convert images into suitable file for FPGA to use

Please execute the first block 

In [1]:
from PIL import Image
from collections import Counter
from scipy.spatial import KDTree
import numpy as np








Please enter the name of the original picture including the notion of "jpg" or "png"

In [5]:
filename = input("What's the image name?")

What's the image name?Init_spite.png


Please specify the palette you are going to use, if you want to use the standard 256 color, execute the last block at the bottom of the jupyter notebook and the second block below; 

if you want to use the palette that you designed, please modify the first block on you need and execute it. Remember!!! If you are using the Palette_hex that you designed, you also need to modify the Palette file in the Quartus. Each "8'dxx" is a index which is the same index of your color in the "Palette_hex" array in python code


In [11]:
palette_hex = ['0x000000', '0x0000FF', '0x00FF00', '0x00FFFF',  #0-3
               '0xFF0000', '0xFF00FF', '0xFFFF00', '0xFFFFFF',  #4-7
               '0x182D5A', '0xEC4523', '0xFBFEFF', '0xF0F0F0']

You need to make sure that your input image is 640 x 480 to do the next step.

Execute the next step to generate the index file.

It will take about 1 min depends on your computer's performance.

In [11]:
# Generate the Intergate based Index
#

def hex_to_rgb(num):
    h = str(num)
    return int(h[0:4], 16), int(('0x' + h[4:6]), 16), int(('0x' + h[6:8]), 16)
def rgb_to_hex(num):
    h = str(num)
    return int(h[0:4], 16), int(('0x' + h[4:6]), 16), int(('0x' + h[6:8]), 16)

new_w, new_h = 640,480

palette_rgb = [hex_to_rgb(color) for color in palette_hex]

pixel_tree = KDTree(palette_rgb)
im = Image.open("./sprite_originals/" + filename) #Can be many different formats.
im = im.convert("RGBA")
layer = Image.new('RGBA',(new_w, new_h), (0,0,0,0))
layer.paste(im, (0, 0))
im = layer
#im = im.resize((new_w, new_h),Image.ANTIALIAS) # regular resize
pix = im.load()
pix_freqs = Counter([pix[x, y] for x in range(im.size[0]) for y in range(im.size[1])])
pix_freqs_sorted = sorted(pix_freqs.items(), key=lambda x: x[1])
pix_freqs_sorted.reverse()
print(pix)
outImg = Image.new('RGB', im.size, color='white')
outFile = open("./sprite_bytes/" + filename + '_int.txt', 'w')
i = 0
for y in range(im.size[1]):
    for x in range(im.size[0]):
        pixel = im.getpixel((x,y))
        #print(pixel)
        if(pixel[3] < 200):
            outImg.putpixel((x,y), palette_rgb[0])
            outFile.write("%i\n" %(0))
            #print(i)
        else:
            index = pixel_tree.query(pixel[:3])[1]
            outImg.putpixel((x,y), palette_rgb[index])
            outFile.write("%i\n" %(index))
        i += 1
print("Finish!")
outFile.close()
outImg.save("./sprite_converted/" + filename)


Finish!


Next, Depends on where you want to put you data in:

1. Put in to the Resources Ram, i.e the SRAM:

Execute the first block, a "hex" file with the name similar to what you have specified in "filename" will appear in the "Spite_Bytes" file. Go the the control panel and download it.


2. Put in to the On_chip_frame buffer directly (Which I assume you will not use)

if you want to put you data directly in the On_chip memory frame buffer, execute the second block below. The code will directly generate the correct file into the correct location.What you need to do next is goto the Quartus, go the the "assignment" bottom at the top, press something like "Update the memory initilization file". Then if you have alreay compile you file, then only press the "assembler" bottom and regenerate the programming sof file; if you have not yet compile your project, go directly to compile yet.

In [12]:
#Generate the Hex file
#HEX!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!File
Im8_fn = "./sprite_bytes/" + filename + '_int.txt'
Im16_fn = "./sprite_bytes/" + filename + '_16.hex'

Im8_f = open(Im8_fn,'r')
Im16_f = open(Im16_fn,'w')
Im8 = np.array(Im8_f.readlines(),dtype = np.uint16)

Im16_len = len(Im8) >> 1
Im16 = np.zeros(Im16_len,dtype = np.uint16)
print("Im16 has length",len(Im16))

for r in range(0,Im16_len):
    Im16[r] = ((np.uint16(Im8[2*r+1])) << 8) + np.uint16(Im8[2*r])
    Im16_f.write("%04x" %(Im16[r]))

print("HEX File generate success")
Im8_f.close()
Im16_f.close()


Im16 has length 153600
HEX File generate success


In [49]:
#Generate the MIF file
#Put into the On_chip_buffer directly
uf_n = "./db/Bomberman.ram0_ram_153601x16_b57232c3.hdl.mif"
Im8_fn = "./sprite_bytes/" + filename + '_int.txt'

Im8_f = open(Im8_fn,'r')
uf = open(uf_n,'w')

#Write Signature
uf.write("-- begin_signature\n-- On_chip_buffer_v2\n-- end_signature\nWIDTH=16;\nDEPTH=153600;\n")
uf.write("\n")
uf.write("ADDRESS_RADIX=UNS;\nDATA_RADIX=BIN;\n\nCONTENT BEGIN\n")


Im8 = np.array(Im8_f.readlines(),dtype = np.uint16)

Im16_len = len(Im8) >> 1
Im16 = np.zeros(Im16_len,dtype = np.uint16)
print("Im16 has length",len(Im16))

for r in range(153599,0,-1):
    Im16[r] = ((np.uint16(Im8[2*r])) << 8) + np.uint16(Im8[2*r+1])
    uf.write(" ")
    uf.write("%i: " %r)
    uf.write(" ")
    uf.write(bin(Im16[r])[2:].zfill(16))
    uf.write(";\n")

Im16[0] = ((np.uint16(Im8[0])) << 8) + np.uint16(Im8[1])
uf.write(" ")
uf.write("0: ")
uf.write(" ")
uf.write(bin(Im16[0])[2:].zfill(16))
uf.write(";\n")
  
uf.write("END;")

uf.close()
Im8_f.close()
print("Finish")



Im16 has length 153600
Finish


In [47]:
#Generate the txt File
#Result writen in hex
Im8_fn = "./sprite_bytes/" + filename + '_int.txt'
Im16_fn = "./sprite_bytes/" + filename + '_16.txt'

Im8_f = open(Im8_fn,'r')
Im16_f = open(Im16_fn,'w')
Im8 = np.array(Im8_f.readlines(),dtype = np.uint16)

Im16_len = len(Im8) >> 1
Im16 = np.zeros(Im16_len,dtype = np.uint16)
print("Im16 has length",len(Im16))

for r in range(0,Im16_len):
    Im16[r] = ((np.uint16(Im8[2*r])) << 8) + np.uint16(Im8[2*r+1])
    Im16_f.write("%04x\n" %(Im16[r]))

print("TXT File generate success")
Im8_f.close()
Im16_f.close()


Im16 has length 153600
TXT File generate success


# Jerry's Special Functions

The following functions are only used for hardware debugging which eric you will not use.

In [57]:
#Generate the dat File
Im8_fn = "./sprite_bytes/" + filename + '_int.txt'
Im16_fn = "./sprite_bytes/" + filename + '_16.dat'

Im8_f = open(Im8_fn,'r')
Im16_f = open(Im16_fn,'w')
Im8 = np.array(Im8_f.readlines(),dtype = np.uint16)

Im16_len = len(Im8) >> 1
Im16 = np.zeros(Im16_len,dtype = np.uint16)
print("Im16 has length",len(Im16))

for r in range(0,Im16_len):
    Im16[r] = ((np.uint16(Im8[2*r])) << 8) + np.uint16(Im8[2*r+1])
    Im16_f.write("@%x " %r)
    Im16_f.write("%04x\n" %(Im16[r]))

print("DAT File generate success")
Im8_f.close()
Im16_f.close()


Im16 has length 153600
DAT File generate success


In [47]:
#Generate the standard patellte document


std_hex_name = '.\Patettes\std_raw_hex.txt'
std_hex = open(std_hex_name,'r')

std_p = open('.\Patettes\std_p.txt','w')
std_pl = np.array(std_hex.readlines())

vector = np.zeros(256,dtype = np.uint32)
counter = 0

for r in std_pl:
    for c in range(0,len(r)):
        if(r[c] == '#'):
            vector[counter] = (int(r[c+1],16)<<20) + (int(r[c+2],16)<<16) + (int(r[c+3],16) <<12) + (int(r[c+4],16) <<8) + (int(r[c+5],16) << 4) + int(r[c+6],16)
            counter = counter + 1
            break
    
for r in vector:
    std_p.write("'0x")
    std_p.write("%06x'," %r)
    std_p.write("\n")
    
std_p.close()
std_hex.close()

# Standard Palettes

This is the standard Palette file that contains a 256 types of common 24-bit color.

In [3]:
palette_hex = [
'0x000000',
'0x800000',
'0x008000',
'0x808000',
'0x000080',
'0x800080',
'0x008080',
'0xc0c0c0',
'0x808080',
'0xff0000',
'0x00ff00',
'0xffff00',
'0x0000ff',
'0xff00ff',
'0x00ffff',
'0xffffff',
'0x000000',
'0x00005f',
'0x000087',
'0x0000af',
'0x0000d7',
'0x0000ff',
'0x005f00',
'0x005f5f',
'0x005f87',
'0x005faf',
'0x005fd7',
'0x005fff',
'0x008700',
'0x00875f',
'0x008787',
'0x0087af',
'0x0087d7',
'0x0087ff',
'0x00af00',
'0x00af5f',
'0x00af87',
'0x00afaf',
'0x00afd7',
'0x00afff',
'0x00d700',
'0x00d75f',
'0x00d787',
'0x00d7af',
'0x00d7d7',
'0x00d7ff',
'0x00ff00',
'0x00ff5f',
'0x00ff87',
'0x00ffaf',
'0x00ffd7',
'0x00ffff',
'0x5f0000',
'0x5f005f',
'0x5f0087',
'0x5f00af',
'0x5f00d7',
'0x5f00ff',
'0x5f5f00',
'0x5f5f5f',
'0x5f5f87',
'0x5f5faf',
'0x5f5fd7',
'0x5f5fff',
'0x5f8700',
'0x5f875f',
'0x5f8787',
'0x5f87af',
'0x5f87d7',
'0x5f87ff',
'0x5faf00',
'0x5faf5f',
'0x5faf87',
'0x5fafaf',
'0x5fafd7',
'0x5fafff',
'0x5fd700',
'0x5fd75f',
'0x5fd787',
'0x5fd7af',
'0x5fd7d7',
'0x5fd7ff',
'0x5fff00',
'0x5fff5f',
'0x5fff87',
'0x5fffaf',
'0x5fffd7',
'0x5fffff',
'0x870000',
'0x87005f',
'0x870087',
'0x8700af',
'0x8700d7',
'0x8700ff',
'0x875f00',
'0x875f5f',
'0x875f87',
'0x875faf',
'0x875fd7',
'0x875fff',
'0x878700',
'0x87875f',
'0x878787',
'0x8787af',
'0x8787d7',
'0x8787ff',
'0x87af00',
'0x87af5f',
'0x87af87',
'0x87afaf',
'0x87afd7',
'0x87afff',
'0x87d700',
'0x87d75f',
'0x87d787',
'0x87d7af',
'0x87d7d7',
'0x87d7ff',
'0x87ff00',
'0x87ff5f',
'0x87ff87',
'0x87ffaf',
'0x87ffd7',
'0x87ffff',
'0xaf0000',
'0xaf005f',
'0xaf0087',
'0xaf00af',
'0xaf00d7',
'0xaf00ff',
'0xaf5f00',
'0xaf5f5f',
'0xaf5f87',
'0xaf5faf',
'0xaf5fd7',
'0xaf5fff',
'0xaf8700',
'0xaf875f',
'0xaf8787',
'0xaf87af',
'0xaf87d7',
'0xaf87ff',
'0xafaf00',
'0xafaf5f',
'0xafaf87',
'0xafafaf',
'0xafafd7',
'0xafafff',
'0xafd700',
'0xafd75f',
'0xafd787',
'0xafd7af',
'0xafd7d7',
'0xafd7ff',
'0xafff00',
'0xafff5f',
'0xafff87',
'0xafffaf',
'0xafffd7',
'0xafffff',
'0xd70000',
'0xd7005f',
'0xd70087',
'0xd700af',
'0xd700d7',
'0xd700ff',
'0xd75f00',
'0xd75f5f',
'0xd75f87',
'0xd75faf',
'0xd75fd7',
'0xd75fff',
'0xd78700',
'0xd7875f',
'0xd78787',
'0xd787af',
'0xd787d7',
'0xd787ff',
'0xd7af00',
'0xd7af5f',
'0xd7af87',
'0xd7afaf',
'0xd7afd7',
'0xd7afff',
'0xd7d700',
'0xd7d75f',
'0xd7d787',
'0xd7d7af',
'0xd7d7d7',
'0xd7d7ff',
'0xd7ff00',
'0xd7ff5f',
'0xd7ff87',
'0xd7ffaf',
'0xd7ffd7',
'0xd7ffff',
'0xff0000',
'0xff005f',
'0xff0087',
'0xff00af',
'0xff00d7',
'0xff00ff',
'0xff5f00',
'0xff5f5f',
'0xff5f87',
'0xff5faf',
'0xff5fd7',
'0xff5fff',
'0xff8700',
'0xff875f',
'0xff8787',
'0xff87af',
'0xff87d7',
'0xff87ff',
'0xffaf00',
'0xffaf5f',
'0xffaf87',
'0xffafaf',
'0xffafd7',
'0xffafff',
'0xffd700',
'0xffd75f',
'0xffd787',
'0xffd7af',
'0xffd7d7',
'0xffd7ff',
'0xffff00',
'0xffff5f',
'0xffff87',
'0xffffaf',
'0xffffd7',
'0xffffff',
'0x080808',
'0x121212',
'0x1c1c1c',
'0x262626',
'0x303030',
'0x3a3a3a',
'0x444444',
'0x4e4e4e',
'0x585858',
'0x626262',
'0x6c6c6c',
'0x767676',
'0x808080',
'0x8a8a8a',
'0x949494',
'0x9e9e9e',
'0xa8a8a8',
'0xb2b2b2',
'0xbcbcbc',
'0xc6c6c6',
'0xd0d0d0',
'0xdadada',
'0xe4e4e4',
'0x14006C'
]

In [64]:
palette_hex = [
'0x000000',
'0xE7EFDC',    
'0xE6E6E6',
'0xE7E7E7',
'0xF6E3DA',
'0xE3EFE3',
'0xEDEEEA',
'0xE5EDF7',
'0xF5EEEB',
'0xF0F0F0',
'0xEDF5F2',
'0xF3F3F0',
'0xF2F2F2',
'0xE9F5FE',
'0xF4F4F4',
'0xF9FBEE',
'0xEFF8FE',
'0xF7F7F7',
'0x2A2A2A',
'0x3F3F3F',
'0x5F5F5F',
'0x818181',
'0x828282',
'0xA3A3A3',
'0xD8D8D8',
'0xDCDCDC',
'0xE0E0E0',
'0xE3E2E2',
'0xABACAC',
'0xCDCECE',
'0xEAE4E1',
'0xE6DAD2',
'0xEADFDB',
'0xEAD5CA',
'0xF6DCD2',
'0xF4C0AB',
'0xFD894B',
'0xF8D2BC',
'0xFBA277',
'0xFD9967',
'0xFBAA85',
'0xB3B495',
'0xD4D5BB',
'0xDEE5BC',
'0xDCDCCE',
'0xE4E4DC',
'0xD6D8C2',
'0xE2E3D4',
'0xDADBC1',
'0xE8EAD6',
'0xEEEED7',
'0xEBEFD4',
'0xE2E9CA',
'0xE5EDD3',
'0x47A959',
'0x6CB977',
'0x83C38A',
'0x99D19B',
'0xC9E5CB',
'0xA0A17E',
'0xC3DBA6',
'0xD0E0B1',
'0xDAE2B2',
'0x2A5FBF',
'0x4C8EBF',
'0x4D7CCD',
'0x3E8FEC',
'0x5E9BE5',
'0x4294EF',
'0x5CA3EC',
'0x7ACDFA',
'0x71A5E7',
'0x77B7EB',
'0x87BAE0',
'0x7FBEF3',
'0x82C5F9',
'0x9AC7E7',
'0x80D3FA',
'0xB0D1F8',
'0x93BDDC',
'0x839FD6',
'0x85B7DD',
'0xB0CEE4',
'0xB6D4EB',
'0xCCE4F4',
'0xB3DCF7',
'0xB9E2FD',
'0xD9E1F1',
]